In [28]:
import pandas as pd

# crea un 2 df pandas con identicas columnas y filas usanndo from faker import Faker
from faker import Faker

fake = Faker()

data = []
for _ in range(10):
    data.append([fake.first_name(), fake.last_name(), fake.email()])

df1 = pd.DataFrame(data, columns=['first_name', 'last_name', 'email'])

In [42]:
df1.to_csv('df1.csv', index=False)

In [52]:
df2 = pd.read_csv('df1.csv')

In [31]:
import pandas as pd

def comparar_columnas(df1, df2):
    """
    Compara cada columna entre dos dataframes y retorna un DataFrame con los resultados
    
    Parameters:
    -----------
    df1: primer DataFrame
    df2: segundo DataFrame
    
    Returns:
    --------
    DataFrame con los resultados de la comparación
    """
    resultados = []
    
    for columna in df1.columns:
        # Verificar si los valores son iguales
        son_iguales = df1[columna].equals(df2[columna])
        
        # Si no son iguales, contar cuántos valores diferentes hay
        if not son_iguales:
            # Manejo especial para columnas datetime
            if df1[columna].dtype != df2[columna].dtype:
                diferentes = "Tipos diferentes"
            else:
                try:
                    diferentes = (df1[columna] != df2[columna]).sum()
                except:
                    diferentes = "No comparable directamente"
        else:
            diferentes = 0
            
        resultados.append({
            'Columna': columna,
            'Son_Iguales': son_iguales,
            'Tipo_DF1': df1[columna].dtype,
            'Tipo_DF2': df2[columna].dtype,
            'Valores_Diferentes': diferentes
        })
    
    return pd.DataFrame(resultados)

# Ejemplo de uso:
# resultado = comparar_columnas(df1, df2)
# print(resultado)

# Para mostrar solo las columnas con diferencias:
# columnas_diferentes = resultado[~resultado['Son_Iguales']]
# print(columnas_diferentes)

# Para una versión más simple que solo muestre si son iguales:
def comparacion_simple(df1, df2):
    """
    Versión simplificada que solo muestra si las columnas son iguales o no
    """
    for columna in df1.columns:
        son_iguales = df1[columna].equals(df2[columna])
        print(f"{columna}: {'Iguales' if son_iguales else 'Diferentes'}")



In [53]:
# Comparacion simple de columnas
comparacion_simple(df1, df2)

first_name: Diferentes
last_name: Iguales
email: Iguales


In [54]:
comparar_columnas(df1, df2)

,Columna,Son_Iguales,Tipo_DF1,Tipo_DF2,Valores_Diferentes
0,first_name,False,object,object,1
1,last_name,True,object,object,0
2,email,True,object,object,0


In [50]:
import pandas as pd

def identificar_registros_diferentes(df1, df2, columna):
    """
    Identifica y muestra los registros diferentes entre dos DataFrames para una columna específica
    
    Parameters:
    -----------
    df1: primer DataFrame
    df2: segundo DataFrame
    columna: nombre de la columna a comparar
    
    Returns:
    --------
    DataFrame con los registros diferentes
    """
    # Crear índice para rastrear los registros
    df1_temp = df1.copy()
    df2_temp = df2.copy()
    
    # Agregar índice original como columna
    df1_temp['indice_original'] = df1_temp.index
    df2_temp['indice_original'] = df2_temp.index
    
    # Identificar registros diferentes
    mascara_diferencias = df1_temp[columna] != df2_temp[columna]
    
    # Crear DataFrame con las diferencias
    diferencias = pd.DataFrame({
        'Indice': df1_temp[mascara_diferencias]['indice_original'],
        f'{columna}_DF1': df1_temp[mascara_diferencias][columna],
        f'{columna}_DF2': df2_temp[mascara_diferencias][columna]
    })
    
    # Agregar información adicional si existe en los DataFrames originales
    # Por ejemplo, si hay otras columnas que ayuden a identificar el registro
    columnas_identificacion = ['DRUT_PERSONA', 'NOMB_PATERNO', 'NOMB_NOMBRES']
    for col in columnas_identificacion:
        if col in df1.columns and col != columna:
            diferencias[f'{col}'] = df1_temp[mascara_diferencias][col]
    
    return diferencias

def mostrar_resumen_diferencias(diferencias):
    """
    Muestra un resumen de las diferencias encontradas
    """
    total_diferencias = len(diferencias)
    print(f"\nTotal de registros diferentes: {total_diferencias}")
    print("\nPrimeros 10 registros diferentes:")
    print(diferencias.head(10))
    
    # Análisis básico de las diferencias
    print("\nAnálisis de diferencias:")
    print("-" * 50)
    
    # Mostrar algunos ejemplos de cómo son las diferencias
    if 'DRUT_PERSONA_DF1' in diferencias.columns and 'DRUT_PERSONA_DF2' in diferencias.columns:
        print("\nEjemplos de las diferencias encontradas:")
        for i in range(min(5, len(diferencias))):
            print(f"\nRegistro {i+1}:")
            print(f"DF1: {diferencias.iloc[i]['DRUT_PERSONA_DF1']}")
            print(f"DF2: {diferencias.iloc[i]['DRUT_PERSONA_DF2']}")

# Para guardar los resultados en un archivo Excel:
def guardar_diferencias_excel(diferencias, nombre_archivo='diferencias.xlsx'):
    """
    Guarda las diferencias encontradas en un archivo Excel
    """
    diferencias.to_excel(nombre_archivo, index=False)
    print(f"\nResultados guardados en {nombre_archivo}")

In [55]:
# Ejemplo de uso:
diferencias = identificar_registros_diferentes(df1, df2, 'first_name')
mostrar_resumen_diferencias(diferencias)


Total de registros diferentes: 1

Primeros 10 registros diferentes:
   Indice first_name_DF1 first_name_DF2
8       8          Louis         Louise

Análisis de diferencias:
--------------------------------------------------
